# Motivation
论文原文：[《Behavior Sequence Transformer for E-commerce Recommendation in Alibaba》](https://arxiv.org/abs/1905.06874v1)

参考学习地址：[推荐系统系列（二）：FFM理论与实践](https://zhuanlan.zhihu.com/p/89847045)

BST用于推荐精排阶段，通过MHA捕捉用户行为序列（UBS：User Behavior Sequence）的序列信息。之前大部分UBS建模归结为：
* 朴素简单的sum/mean pooling
* weight pooling。例如经典的DIN使用attention捕捉候选item与用户点击item序列之间的相似性作为weight。（AFM虽然也用了attention-based pooling，但输入非序列数据）
* RNN类，考虑时序信息，例如DIEN中利用GRU捕捉UBS信息

由于Transformer在很多NLP任务中表现超过RNN，且有可并行的优势，所以利用Transformer代替RNN也是一个很自然的idea。

<img style="display: block; margin: 0 auto;" src="../../../assets/images/bst.png" width = "800" height = "600" alt="BST" align=center />

## Transformer Layer
BST是携带了时序位置信息的MHA（相对于AutoInt），用在用户历史点击的item序列上。时序信息的引入简单的使用了Position Embedding，作者尝试使用了sin/cos函数的位置嵌入，效果不好，最终采用对时间戳Embedding的形式：
$$
\begin{align}
pos(v_i)=timestamp(v_t)-timestamp(v_i)
\end{align}
$$
其中$v_i$表示用户点击序列中第i个item的时间戳，$v_t$表示候选推荐的时间戳。并且BST通过<font color="red">item embedding（每个item对应的红色部分）和position embedding（每个item对应的蓝色部分）拼接，而非原transformer中的相加</font>。

Self-Attention的部分延用Transformer如下：
$$
\begin{align}
\mathbf{S}&=MH(\mathbf{E})=Concate(head_1,..,head_h)\mathbf{W}^H\\
head_i&=Attention(\mathbf{EW}_i^Q,\mathbf{EW}_i^K,\mathbf{EW}_i^V)=softmax(\frac{\mathbf{EW}_i^Q\mathbf{EW}_i^K}{\sqrt{d}})\mathbf{EW}_i^V
\end{align}
$$

其中，
* $\mathbf{E}\in \R^{n*d}$ 表示序列中拼接了位置信息后的item embedding矩阵，$n$ 为序列长度，$d$ 表示嵌入维度
* $\mathbf{W}_i^Q,\mathbf{W}_i^K,\mathbf{W}_i^V\in \R^{d*d}$分别表示第$i$个head的query，key，value转换矩阵
* $\mathbf{W}^H\in \R^{hd*d}$ 表示拼接多头输出后的转换矩阵，注意最终输出 $\mathbf{S}\in \R^{n*d}$ 和 $\mathbf{E}$ 维度是相同的

Point-wise feed-forward network也即在$\mathbf{S}$后加一些FFN小网络增强非线性，注意这里的网络是point-wise，即**操作粒度**$\mathbf{S}\in \R^{n*d}$ 中的每一个embedding（每一行）
$$
\begin{align}
\mathbf{S'}&=LayerNorm(\mathbf{S}+Dropout(MH(\mathbf{S})))\\
\mathbf{F}&=LayerNorm(\mathbf{S'}+Drouput(LeakyReLU(\mathbf{S'}\mathbf{W}^{(1)}+b^{(1)})\mathbf{W}^{(2)}+b^{(2)}))
\end{align}
$$

# 一些思考
* 使用concat(item, position)而非sum，原因分析：
  * 保持信息独立性，sum表示位置和内容信息在**同一向量空间混合**，而concat代表在**不同向量空间保持独立**，也即BST认为物品的语义与位置信息解耦。e.g.序列：A->B->A，相加的话认为两次A的表示不同，而拼接则认为语义embedding始终相同，位置单独编码。而在NLP中内容与位置由于语法约束，所以呈现强相关。
  * 位置建模灵活性，推荐中位置可能有复杂效应，e.g. 更近的位置更重要；特定位置有特殊意义；相对位置比绝对位置更重要（RoPE？）。